In [47]:
import pandas as pd
import geopandas as gpd

In [48]:

#from sqlalchemy import text

#from nvi_etl import db_engine

#parcels_q = text("""
#SELECT *
#FROM raw.detodp_assessor_20240205
#""")

#parcels_df = gpd.read_postgis(parcels_q, db_engine)

In [ ]:
tax = pd.read_csv("V:/IPDS/Wayne County Tax Foreclosures/Data/2024/Prepared/wcto_foreclosed_05092024.csv")
detodp = pd.read_csv("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/detodp_assessor_20240205.csv")
# As I couldnt get the data from EDW directly, I downloaded the data from the EDW and placed on the Local Drive)

C:\Users\abhi\AppData\Local\Temp\ipykernel_23624\1060489119.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  detodp = pd.read_csv("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/detodp_assessor_20240205.csv")


In [50]:
from shapely import wkb

# Convert WKB to Geometry
detodp["geometry"] = detodp["geom"].apply(wkb.loads)

# Convert to a GeoDataFrame
detodp_gdf = gpd.GeoDataFrame(detodp, geometry="geometry", crs="EPSG:4326")

# Drop the old WKB column if not needed
detodp_gdf.drop(columns=["geom"], inplace=True)

In [51]:
print(tax.head())

    PARCEL_ID         STREET     CITY STATE    ZIP
0  01002114-5  210 E BETHUNE  DETROIT    MI  48202
1  01002294-5    430 MARSTON  DETROIT    MI  48202
2   01002353.  438 MT VERNON  DETROIT    MI  48202
3   01002685.      313 HAGUE  DETROIT    MI  48202
4   01002812.   424 HOLBROOK  DETROIT    MI  48202


In [52]:
print(detodp.head())

   id                                               geom  objectid  object_id  \
0   1  0106000020E61000000100000001030000000100000005...         1      77483   
1   2  0106000020E61000000100000001030000000100000005...         2      77500   
2   3  0106000020E61000000100000001030000000100000005...         3      77523   
3   4  0106000020E61000000100000001030000000100000005...         4      77533   
4   5  0106000020E61000000100000001030000000100000009...         5      77555   

   parcel_num  ward           address council_di zip_code  \
0   06004253.   6.0      1699 KENDALL          2    48238   
1   22038234.  22.0  17555 STRATHMOOR          2    48235   
2   22034773.  22.0     17592 ARDMORE          2    48235   
3  22030567-8  22.0    18065 SCHAEFER          2    48235   
4   22036058.  22.0    18451 FREELAND          2    48235   

                           taxpayer_1  ...  assessed_v taxable_va landmap  \
0         DETROIT LAND BANK AUTHORITY  ...           0          0    

In [53]:
#Filter City for only Detroit
tax = tax[tax["CITY"] == 'DETROIT']

In [54]:
def fix_parcel_id(parcel_id: str):
    if "." in parcel_id:
        return parcel_id
    if "-" in parcel_id:
        return parcel_id
    return parcel_id.zfill(8) + "."

# Apply the function to the PARCEL_ID column
tax = tax.dropna(subset=["PARCEL_ID"])
tax["PARCEL_ID"] = tax["PARCEL_ID"].astype(str).apply(fix_parcel_id)


In [56]:
# Merge the two dataframes
merged = pd.merge(tax, detodp, left_on="PARCEL_ID", right_on="parcel_num", how="inner")
print(merged)

       PARCEL_ID             STREET     CITY STATE    ZIP      id  \
0     01002114-5      210 E BETHUNE  DETROIT    MI  48202  307107   
1     01002294-5        430 MARSTON  DETROIT    MI  48202   62414   
2      01002353.      438 MT VERNON  DETROIT    MI  48202  288862   
3      01002685.          313 HAGUE  DETROIT    MI  48202  193292   
4      01002812.       424 HOLBROOK  DETROIT    MI  48202    2589   
...          ...                ...      ...   ...    ...     ...   
2094   14010947.          4156 28TH  DETROIT    MI  48210  127504   
2095   14011076.          6100 28TH  DETROIT    MI  48210  186490   
2096   14011764.          4924 30TH  DETROIT    MI  48210  115795   
2097   14012041.   9748 BROADSTREET  DETROIT    MI  48204  221974   
2098  14012215-6  12001 BROADSTREET  DETROIT    MI  48204  353170   

                                                   geom  objectid  object_id  \
0     0106000020E61000000100000001030000000100000005...    307107     372749   
1     01060

In [57]:
nvi_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/NVI Zones/nvi_neighborhood_zones_temp_2025.shp")
detroit_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/City_of_Detroit_Boundary/City_of_Detroit_Boundary.shp")
cd_shp = gpd.read_file("P:/2024_Projects/NVI24/Development/Workspace/Abhi Workspace/Secondary Data Pull/Detroit_City_Council_Districts_2026/Detroit_City_Council_Districts_2026.shp")


In [58]:
from shapely import wkb

# Convert WKB to Geometry
merged["geometry"] = merged["geom"].apply(wkb.loads)

# Convert to a GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged, geometry="geometry", crs="EPSG:4326")

# Drop the old WKB column if not needed
merged_gdf.drop(columns=["geom"], inplace=True)

#### City of Detroit

In [59]:
# Ensure both GeoDataFrames have the same CRS
merged_gdf = merged_gdf.to_crs(detroit_shp.crs)

# Perform a spatial join to assign births to geographic areas
merged_with_geography = gpd.sjoin(merged_gdf, detroit_shp, how="left", predicate="within")

In [60]:
detodp_gdf = detodp_gdf.to_crs(detroit_shp.crs)
detodp_with_geography = gpd.sjoin(detodp_gdf, detroit_shp, how="left", predicate="within")

In [66]:
# Total number of rows (properties in foreclosure)
num_foreclosures = len(merged_with_geography)
print(f"Number of properties in foreclosure: {num_foreclosures}")

# Total number of properties (from detodp with geography)
total_properties = len(detodp_with_geography)
print(f"Total number of properties: {total_properties}")

# Calculate foreclosure rate by geographic area
foreclosure_rate = (num_foreclosures / total_properties) * 100
print(f"% of properties in foreclosure for taxes: {foreclosure_rate}")

Number of properties in foreclosure: 2099
Total number of properties: 381265
% of properties in foreclosure for taxes: 0.5505357166275425


### NVI Zones

In [71]:
merged_nvi = merged_gdf.to_crs(nvi_shp.crs)
merged_with_nvi = gpd.sjoin(merged_nvi, nvi_shp, how="left", predicate="within")
detodp_nvi= detodp_gdf.to_crs(nvi_shp.crs)
detodp_with_nvi = gpd.sjoin(detodp_nvi, nvi_shp, how="left", predicate="within")

In [82]:
# Total number of properties in foreclosure, grouped by district_n and zone_id
num_foreclosures_nvi = merged_with_nvi.groupby(["district_n", "zone_id"]).size().reset_index(name="num_foreclosures")

# Total number of properties (from detodp with geography), grouped by district_n and zone_id
total_properties_nvi = detodp_with_nvi.groupby(["district_n", "zone_id"]).size().reset_index(name="total_properties")

foreclosure_rate_nvi = num_foreclosures_nvi.merge(total_properties_nvi, on=["district_n", "zone_id"], how="left")
foreclosure_rate_nvi["foreclosure_rate_nvi"] = (foreclosure_rate_nvi["num_foreclosures"] / foreclosure_rate_nvi["total_properties"]) * 100

display(foreclosure_rate_nvi)

,district_n,zone_id,num_foreclosures,total_properties,foreclosure_rate_nvi
0,1,1a,80,16870,0.474215
1,1,1b,68,15165,0.448401
2,1,1c,97,16990,0.570924
3,2,2a,89,18422,0.483118
4,2,2b,34,10570,0.321665
5,2,2c,137,15526,0.882391
6,3,3a,197,28045,0.702443
7,3,3b,148,16550,0.894260
8,3,3c,95,22114,0.429592
9,4,4a,176,24918,0.706317


### CD 2026

In [83]:
merged_cd = merged_gdf.to_crs(cd_shp.crs)
merged_with_cd = gpd.sjoin(merged_cd, cd_shp, how="left", predicate="within")
detodp_cd= detodp_gdf.to_crs(cd_shp.crs)
detodp_with_cd = gpd.sjoin(detodp_cd, cd_shp, how="left", predicate="within")

In [85]:
# Total number of properties in foreclosure, grouped by district_n and zone_id
num_foreclosures_cd = merged_with_cd.groupby("district_1").size().reset_index(name="num_foreclosures")

# Total number of properties (from detodp with geography), grouped by district_n and zone_id
total_properties_cd = detodp_with_cd.groupby("district_1").size().reset_index(name="total_properties")

foreclosure_rate_cd = num_foreclosures_cd.merge(total_properties_cd, on="district_1", how="left")
foreclosure_rate_cd["foreclosure_rate_cd"] = (foreclosure_rate_cd["num_foreclosures"] / foreclosure_rate_cd["total_properties"]) * 100

display(foreclosure_rate_cd)

,district_1,num_foreclosures,total_properties,foreclosure_rate_cd
0,District 1,245,49032,0.499674
1,District 2,262,44627,0.587089
2,District 3,440,66727,0.659403
3,District 4,378,64293,0.587933
4,District 5,297,53136,0.558943
5,District 6,171,52463,0.325944
6,District 7,304,50324,0.604086
